In [18]:
import os
import pandas as pd
import numpy as np

OUTPUT_DIR = "outputs"

# Carregando os arquivos CSV com os resultados das etapas anteriores
df_T2  = pd.read_csv(os.path.join(OUTPUT_DIR, "T2_saldo.csv"))
df_T3  = pd.read_csv(os.path.join(OUTPUT_DIR, "T3_limite.csv"))
df_T5  = pd.read_csv(os.path.join(OUTPUT_DIR, "T5_risk_regiao.csv"))
df_T6  = pd.read_csv(os.path.join(OUTPUT_DIR, "T6_risk_valor.csv"))
df_T7  = pd.read_csv(os.path.join(OUTPUT_DIR, "T7_risk_horario.csv"))
df_T8  = pd.read_csv(os.path.join(OUTPUT_DIR, "T8_risk_agg.csv"))
df_T9  = pd.read_csv(os.path.join(OUTPUT_DIR, "T9_combined.csv"))
df_T10 = pd.read_csv(os.path.join(OUTPUT_DIR, "T10_user_check.csv"))
df_L1  = pd.read_csv(os.path.join(OUTPUT_DIR, "L1_transacoes_aprovacao.csv"))
df_L2  = pd.read_csv(os.path.join(OUTPUT_DIR, "L2_usuarios_atualizados.csv"))

# Verifica se nenhuma transação foi aprovada com saldo insuficiente
print("\nVerificando T2 (saldo)...")
assert all(col in df_T2.columns for col in ["saldo", "valor_transacao", "aprovacao"]), \
    "Colunas esperadas não encontradas em T2"

mask_incons_T2 = (
    df_T2["saldo"].notna() & df_T2["valor_transacao"].notna() &
    (df_T2["saldo"] < df_T2["valor_transacao"]) &
    (df_T2["aprovacao"] == True)
)
if mask_incons_T2.any():
    print("Há transações com saldo insuficiente que foram aprovadas:")
    print(df_T2.loc[mask_incons_T2, ["id_transacao", "saldo", "valor_transacao"]].head())
else:
    print("Tudo certo em T2.")

# Verifica se os limites por modalidade foram respeitados nas transações aprovadas
print("\nVerificando T3 (limites por modalidade)...")
required_cols = ["id_transacao", "modalidade_pagamento", "valor_transacao", "aprovacao"]
assert all(col in df_T3.columns for col in required_cols), "Colunas faltantes em T3"

incons_T3 = []
for _, row in df_T3.iterrows():
    if row["aprovacao"]:
        lim_col = f"limite_{row['modalidade_pagamento']}"
        if lim_col not in row or pd.isna(row[lim_col]):
            inconsistência = f"Transação {row['id_transacao']}: limite ausente para {row['modalidade_pagamento']}"
            inconsistência += f" | valor: {row['valor_transacao']}"
            incons_T3.append(inconsistência)
        elif row["valor_transacao"] > row[lim_col]:
            incons_T3.append(row["id_transacao"])

if incons_T3:
    print(f"Foram encontradas {len(incons_T3)} transações com problemas de limite.")
    print("Exemplo:", incons_T3[:5])
else:
    print("Tudo certo em T3.")

# Verifica se a combinação entre regras determinísticas e de risco foi feita corretamente
print("\nVerificando T9 (combinação entre regras e risco)...")
merged_3_8 = df_T3[["id_transacao", "aprovacao"]].merge(
    df_T8[["id_transacao", "aprovacao_risco"]],
    on="id_transacao", how="inner"
)
merged_3_8["esperado"] = merged_3_8["aprovacao"] & merged_3_8["aprovacao_risco"]
df_T9_check = df_T9.set_index("id_transacao").loc[merged_3_8["id_transacao"]]
diff_mask = df_T9_check["aprovacao"].values != merged_3_8["esperado"].values

if diff_mask.any():
    print(f"Foram encontradas {diff_mask.sum()} divergências entre o valor esperado e o valor armazenado.")
    print(merged_3_8[diff_mask].head())
else:
    print("Tudo certo em T9.")

# Verifica se os usuários não gastaram mais do que tinham de saldo
print("\nVerificando T10 (total gasto por usuário versus saldo)...")
soma_aprov = df_T10[df_T10["aprovacao"]].groupby("id_usuario")["valor_transacao"].sum()
saldos_originais = df_T10.groupby("id_usuario")["saldo"].first()
mask_violado = soma_aprov > saldos_originais.reindex(soma_aprov.index)

if mask_violado.any():
    print("Usuários com transações aprovadas que somam mais que o saldo original:")
    print(mask_violado[mask_violado].head())
else:
    print("Tudo certo em T10.")

# Verifica se não houve saldo negativo após atualizar os saldos dos usuários
print("\nVerificando L2 (saldos finais não negativos)...")
if (df_L2["saldo"] < 0).any():
    print(f"Encontrados { (df_L2['saldo'] < 0).sum() } saldos negativos.")
    print(df_L2[df_L2["saldo"] < 0].head())
else:
    print("Todos os saldos finais são positivos.")

# Confere se os scores de risco estão todos normalizados entre 0 e 1
print("\nVerificando normalização dos scores...")
assert df_T5["score_regiao"].between(0, 1).all(), "score_regiao fora do intervalo [0,1]"
assert df_T6["score_valor"].between(0, 1).all(), "score_valor fora do intervalo [0,1]"
assert df_T7["score_horario"].between(0, 1).all(), "score_horario fora do intervalo [0,1]"
assert df_T7["hour"].between(0, 23).all(), "Horário fora do intervalo [0,23]"
print("Todos os scores estão dentro dos intervalos esperados.")


# Verificações extras
print("\nProcurando valores anômalos ou inválidos...")
assert (df_T2["valor_transacao"] > 0).all(), "Transações com valor <= 0 em T2"
print("Todas as transações têm valores positivos.")

# Verifica se os identificadores são únicos nos arquivos principais
print("\nVerificando unicidade dos IDs...")
assert df_T2["id_transacao"].is_unique, "ID de transação duplicado em T2"
assert df_T9["id_transacao"].is_unique, "ID duplicado em T9"
assert df_L1["id_transacao"].is_unique, "ID duplicado em L1"
assert not df_L1["id_transacao"].duplicated().any(), "IDs duplicados em L1"
assert df_L2["id_usuario"].is_unique, "ID de usuário duplicado em L2"
print("IDs verificados e sem duplicações.")

# Fim das verificações
print("\nTodas as verificações foram concluídas com sucesso.")


Verificando T2 (saldo)...
Tudo certo em T2.

Verificando T3 (limites por modalidade)...
Tudo certo em T3.

Verificando T9 (combinação entre regras e risco)...
Tudo certo em T9.

Verificando T10 (total gasto por usuário versus saldo)...
Tudo certo em T10.

Verificando L2 (saldos finais não negativos)...
Todos os saldos finais são positivos.

Verificando normalização dos scores...
Todos os scores estão dentro dos intervalos esperados.

Procurando valores anômalos ou inválidos...
Todas as transações têm valores positivos.

Verificando unicidade dos IDs...
IDs verificados e sem duplicações.

Todas as verificações foram concluídas com sucesso.


In [19]:
# Verifica se os mesmos IDs de transações aparecem em L1 e T9
print("\nVerificando consistência entre L1 e T9...")
ids_L1 = set(df_L1["id_transacao"])
ids_T9 = set(df_T9["id_transacao"])
assert ids_L1 == ids_T9, f"Diferença entre IDs em L1 e T9: {len(ids_L1.symmetric_difference(ids_T9))} discrepâncias"
print("IDs em L1 e T9 estão consistentes.")

# Checa se existem transações duplicadas nas principais tabelas
print("\nVerificando duplicatas gerais...")
for df, name in [(df_T2, "T2"), (df_T3, "T3"), (df_T9, "T9"), (df_L1, "L1")]:
    dups = df["id_transacao"].duplicated().sum()
    assert dups == 0, f"{dups} duplicatas encontradas na tabela {name}"
print("Nenhuma duplicata encontrada nas tabelas principais.")

# Garante que as colunas importantes estão completamente preenchidas
print("\nVerificando ausência de valores ausentes em colunas importantes...")
for df, cols, name in [
    (df_T2, ["valor_transacao", "saldo", "aprovacao"], "T2"),
    (df_T3, ["modalidade_pagamento", "valor_transacao"], "T3"),
    (df_T7, ["data_horario", "hour"], "T7"),
    (df_T8, ["score_regiao", "score_valor", "score_horario", "score_total", "aprovacao_risco"], "T8"),
]:
    for col in cols:
        n_nulos = df[col].isna().sum()
        assert n_nulos == 0, f"{name}: coluna {col} tem {n_nulos} valores nulos"
print("Todas as colunas importantes estão completas.")

#  Confirma que os valores de saldo e transações estão dentro de faixas válidas
print("\nVerificando se valores de saldo e transações estão coerentes...")
assert (df_T2["valor_transacao"] > 0).all(), "Existem transações com valor menor ou igual a zero em T2"
assert (df_T2["saldo"] >= 0).all(), "Há saldos negativos em T2"
assert (df_L2["saldo"] >= 0).all(), "Saldos finais negativos encontrados em L2"
print("Os valores estão dentro das faixas esperadas.")

# Garante que nenhum usuário sem saldo tenha tido transações aprovadas
print("\nVerificando se usuários com saldo zero receberam aprovações...")
usuarios_sem_saldo = set(df_L2[df_L2["saldo"] == 0]["id_usuario"])
usuarios_aprovados = set(df_T10[df_T10["aprovacao"]]["id_usuario"])
violadores = usuarios_sem_saldo & usuarios_aprovados
if violadores:
    print(f"{len(violadores)} usuários têm saldo final igual a zero, mas tiveram transações aprovadas.")
else:
    print("Tudo certo: nenhum usuário sem saldo teve transações aprovadas.")

# Verifica se a soma dos scores parciais bate com o score total
print("\nVerificando se o score_total é a soma dos componentes em T8...")
score_check = (
    df_T8["score_total"].round(5) ==
    df_T8[["score_regiao", "score_valor", "score_horario"]].sum(axis=1).round(5)
)
assert score_check.all(), f"Score_total diferente da soma dos componentes em {(~score_check).sum()} casos"
print("score_total está corretamente calculado.")

# Garante que nenhuma transação foi perdida entre T2, T9 e L1
print("\nVerificando se todas as transações foram mantidas ao longo do pipeline...")
n_T2 = len(df_T2)
n_T9 = len(df_T9)
n_L1 = len(df_L1)
assert n_T2 == n_T9 == n_L1, f"Número de transações inconsistente: T2={n_T2}, T9={n_T9}, L1={n_L1}"
print("Nenhuma transação se perdeu durante o processo.")

# Conta quantas transações ocorreram em horário noturno
print("\nAnalisando transações feitas entre 0h e 6h...")
noturnas = df_T7[df_T7["hour"] <= 6]
print(f"Total: {len(noturnas)} transações ({len(noturnas)/len(df_T7)*100:.1f}% do total)")

# Verifica quantos usuários fizeram muitas transações
print("\nVerificando usuários com grande volume de transações...")
usuarios_multiplos = df_T10["id_usuario"].value_counts()
print(f"Usuários com mais de 10 transações: {(usuarios_multiplos > 10).sum()}")
print("Frequência de transações por usuário parece dentro do esperado.")




Verificando consistência entre L1 e T9...
IDs em L1 e T9 estão consistentes.

Verificando duplicatas gerais...
Nenhuma duplicata encontrada nas tabelas principais.

Verificando ausência de valores ausentes em colunas importantes...
Todas as colunas importantes estão completas.

Verificando se valores de saldo e transações estão coerentes...
Os valores estão dentro das faixas esperadas.

Verificando se usuários com saldo zero receberam aprovações...
Tudo certo: nenhum usuário sem saldo teve transações aprovadas.

Verificando se o score_total é a soma dos componentes em T8...
score_total está corretamente calculado.

Verificando se todas as transações foram mantidas ao longo do pipeline...
Nenhuma transação se perdeu durante o processo.

Analisando transações feitas entre 0h e 6h...
Total: 29018 transações (29.0% do total)

Verificando usuários com grande volume de transações...
Usuários com mais de 10 transações: 0
Frequência de transações por usuário parece dentro do esperado.


In [20]:

# Mostra um resumo estatístico simples dos scores
print("\nResumo estatístico dos scores de risco:")
for df, col in [(df_T5, "score_regiao"), (df_T6, "score_valor"), (df_T7, "score_horario")]:
    print(f"{col}: mínimo={df[col].min():.2f}, máximo={df[col].max():.2f}, média={df[col].mean():.2f}")


Resumo estatístico dos scores de risco:
score_regiao: mínimo=0.00, máximo=1.00, média=0.45
score_valor: mínimo=0.00, máximo=1.00, média=0.03
score_horario: mínimo=0.00, máximo=1.00, média=0.50


In [21]:
# Mostra a proporção de transações aprovadas e reprovadas
print("\nVerificando distribuição de aprovações (L1)...")
print(df_L1["aprovacao"].value_counts(normalize=True))


Verificando distribuição de aprovações (L1)...
aprovacao
False    0.67176
True     0.32824
Name: proportion, dtype: float64
